# **Genetic Algorithm**
Genetic algorithm is a search and optimization algorithm inspired by the natural selection in the theory of evolution. we use it to find an approximate solution to a problem especially when it's a complex, large optimization problem with an unknown state.


## GA Pipeline:
GA pipeline is a systematic flow of operation that occure in each generation of the GA algorithm. the flow is depicted as follows.

1. **Initialize population:** The goal here is to create/ randomly generate the first initial population of individual genome(which can be bitstring, real numbers, vectors...).
2. **Evaluate Fitness:** This is done by a fitness evaluation function, which measures how good an individual fits towards the solution(higher fitness == better solution).
3. **Selection:** After the fitness of each individual is evaluated individuals are selected to become parents via a selection strategy. this will ensure '*Survival of the fittest*'.
4. **Crossover:** In this step offsprings are created by combining the genetic material of the selected parents. (Types include single/two point, uniform, Simulated Binary Crossover).
5. **Mutation:** Then the genetic material of the children will be mutated (creating a random genetic variation). They will be mutated with a small probability.
6. **Form new population:** After that a new population with be formed by creating a new generation and by keeping the '*Elites*' (The best solution from current generation to ensure continuity of strong solutions).
7. **Termination Check:** This step will check weather to stop or continue the generation process, if the condition is not satisfied the process will repeat it self from step 2. Some of the conditions to stop are:

>> A. Reached max number of generations.

>> B. Fitness plateaus.

>> C. Found satisfactory solution.

## **GA Parameters in the code:**
The following are the parameters used in the code provided:

1. **POP_SIZE:** Population size which will affect diversity and runtime.
2. **GENES:** Total number of genes an individual has which is '8'.
3. **GENERATIONS:** Number of generations the GA will run, which will affect how long the evolution lasts in our case '30'.
4. **ELITE_COUNT:** This tells the program how many elites to keep and pass onto the next generation.
5. **CROSSOVER_RATE:** The probability of parents performing crossover, which in ourcase is high favoring exploration.
6. **MUTATION_RATE:** Propability to mutate each gene, used to maintain diversity.
7. **INITIAL_MUTATION_STD:** Standard deviation of gaussian noise added during mutation
8. **MUTATION_DECAY:** reduces the standard deviation each generation making the search fine-tunned over time.
9. **INITIAL_SBX_ETA:** Controlls how similar children are to parents in SBX.
10. **SBX_ETA_GROWTH:** Gradually increases eta over generations, making crossover more conservative as search progresses.

## **Fitness Function:**
A fitness function is a function used to evaluate how good an individual(solution) is with  respect to it's optimization goals. Now the code implements this function using two concepts *Emergence* and *Contribution*.

* Emergence: This concept dictates how much of the selected candidate *Emerges* in the individual genes of both inputs.
`emergence = [c - max(a, b) for c, a, b in zip(candidate, INPUT_A, INPUT_B)]`. Now if the candidate is not emergent or in simple terms the `max(a, b)` is greater than the candidate it will yield a negative value which is clamped to 0 saying that it doesn't contribute `emergence = [max(0, e) for e in emergence]
`

* Contribution: Emergence is only rewarder if the parents have minimum support i.e if both parents contribute a base, and the candidate improves upon it which will be considered valuable. `contributions = [min(a, b) * e for a, b, e in zip(INPUT_A, INPUT_B, emergence)]
`

In the final step each contributions are summed and normalized to return a value between [0, 1].

## **Selection Method:**

As discussed above selection is the process of choosing individuals from the current population to become parents for the next generation. Individuals are selected by considering how high the fitness function is and also by having a small probability of individuals with weak fitness function score to join inorder to maintain diversity.

Common selection startegies include Roulette Wheel Selection, Stochastic Acceptance, Tournament Selection, Rank Selection, Stochastic Universal Sampling.

The code implements the Roulette Stochastic Acceptance, which implements an efficient mode of the traditional roulette wheel selection. It picks individuals randomly but selects them based on their fitness.

It first selectes the best fitness value `w_max = max(fitnesses)` which becomes a bench mark for comparision. Then checks if the individual fitness is close to the max
```
if random.random() < fitnesses[i] / w_max:
    return population[i]
```
via the randomly selected picks `i = random.randint(0, len(population) - 1)
`

This is a much more efficient way than roulette wheel selection because it requires computing the sum of all fitnesses and also requires building a cumulative distribution and sampling from it. While the Stochastic acceptance approch only requires the max fitness with out sorting nor the cumulative sum making it faster for real time applications.



## **Crossover**

Crossover simply means combining the genes of the parents to create new children. it helps us to combine the good traits of both parents and explore new search spaces.

Common types of Crossovers' are Single/two point, uniform crossover (For bitstrings), Simulated Binary Crossover (SBX) for real valued genes.

SBX mimics single-point crossover but for real values, it creates offspring that are around the parents' gene values.

The parameter η(eta) controls how close the children are to their parents.
* Low η: More exploration (offspring are spread out)
* High η: More exploitation (offspring stay near parents)

The code Implemets SBX Crossover which decides to crossover by the crossover probability which is `0.9` which is a very high value.
```
if random.random() > CROSSOVER_RATE:
    return p1[:], p2[:]
```
If the condidion is True then it will just return the parents, if not then it will procede with the crossover.

The code also performs a gene by gene crossover by applying probabilistic interpolation of the parents' genes using the value of η. The equations decide how far the children will be from the midpoint of x1 and x2. Then it just clamps the children to ensure values are between (0, 1). The value of `sbx_eta` will increase over time by the rate of `SBX_ETA_GROWTH`.

## **Mutation Strategies**

Mutation is the process of introducing small random change to the indivuduals in the populations with the goal of maintaining diversity and helps escape local optima by also encouraging exploration.

The most common mutation strategies used for real-values are Gaussian (normal) mutation.

The code implements a Gaussian mutation. For each gene in the individual the probability of the gene getting mutated is 10%. if so then, `random.gauss(0, mutation_std)` adds a normally-distributed random number centered at 0. This creates a small nudge up or down in the gene value. Then the induvidual gene is clamped b/n (0, 1). The `mutation_std` is decreased over time `mutation_std *= MUTATION_DECAY`, Which means at earlier stages it will take larger jumps but as the generation progresses it will have smaller tweaks. This is known as adaptive or annealing mutation, a powerful strategy for controlling convergence.

In [5]:
import random
import numpy as np
import math
import time

# === Parameters ===
POP_SIZE = 50
GENES = 8
GENERATIONS = 30
ELITE_COUNT = 3
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.1
INITIAL_MUTATION_STD = 0.5  # High initial value
MUTATION_DECAY = 0.95       # Decay factor per generation
INITIAL_SBX_ETA = 2
SBX_ETA_GROWTH = 1.05       # SBX eta increases each generation

# === Fixed input individuals for blend ===
INPUT_A = [0.9, 0.9, 0.0, 0.8, 0.2, 0.9, 0.7, 0.7]
INPUT_B = [0.8, 0.2, 1.0, 0.3, 0.9, 0.4, 0.6, 0.3]

# === Fixed input individuals for blend ===
# INPUT_A = [1, 0.5, 0.0, 0.4, 1, 0.5, 0.5, 0.4]
# INPUT_B = [0.8, 0.2, 1.0, 0.3, 1, 0.4, 0.6, 0.3]

# === Fitness Function ===
def fitness(candidate):
    emergence = [c - max(a, b) for c, a, b in zip(candidate, INPUT_A, INPUT_B)]
    emergence = [max(0, e) for e in emergence]  # clamp negative emergence to 0
    contributions = [min(a, b) * e for a, b, e in zip(INPUT_A, INPUT_B, emergence)]
    total = sum(contributions)
    return min(total / GENES, 1.0)

# === Initialization ===
def initialize_population():
    return [np.random.uniform(0, 1, GENES).tolist() for _ in range(POP_SIZE)]

# === Selection: Roulette-Wheel via Stochastic Acceptance ===
def roulette_stochastic_acceptance(population, fitnesses, max_tries=100):
    w_max = max(fitnesses)
    tries = 0
    while tries < max_tries: # fixed the infinit loop problem
    # while True:
        i = random.randint(0, len(population) - 1)
        if random.random() < fitnesses[i] / w_max:
            return population[i]
        tries += 1
    # If no individual is selected within max_tries, return a random individual
    return random.choice(population)


# === Crossover: Simulated Binary Crossover (Adaptive Eta) ===
def sbx_crossover(p1, p2, eta):
    if random.random() > CROSSOVER_RATE:
        return p1[:], p2[:]

    child1, child2 = [], []
    for x1, x2 in zip(p1, p2):
        if random.random() <= 0.5:
            if abs(x1 - x2) > 1e-14:
                x1, x2 = min(x1, x2), max(x1, x2)
                rand = random.random()
                beta = 1.0 + (2.0 * (x1) / (x2 - x1))
                alpha = 2.0 - beta ** -(eta + 1)
                if rand <= 1.0 / alpha:
                    betaq = (rand * alpha) ** (1.0 / (eta + 1))
                else:
                    betaq = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta + 1))
                c1 = 0.5 * ((x1 + x2) - betaq * (x2 - x1))
                c2 = 0.5 * ((x1 + x2) + betaq * (x2 - x1))
                child1.append(min(max(c1, 0.0), 1.0))
                child2.append(min(max(c2, 0.0), 1.0))
            else:
                child1.append(x1)
                child2.append(x2)
        else:
            child1.append(x1)
            child2.append(x2)
    return child1, child2

# === Mutation ===
def mutate(individual, mutation_std):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            individual[i] += random.gauss(0, mutation_std)
            individual[i] = min(max(individual[i], 0), 1)  # clip to [0, 1]
    return individual

# === Main GA Loop ===
def genetic_algorithm():
    population = initialize_population()
    mutation_std = INITIAL_MUTATION_STD
    sbx_eta = INITIAL_SBX_ETA

    for gen in range(GENERATIONS):
        fitnesses = [fitness(ind) for ind in population]
        elites = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)[:ELITE_COUNT]
        new_population = [ind for ind, _ in elites]

        while len(new_population) < POP_SIZE:
            parent1 = roulette_stochastic_acceptance(population, fitnesses)
            parent2 = roulette_stochastic_acceptance(population, fitnesses)
            child1, child2 = sbx_crossover(parent1, parent2, eta=sbx_eta)
            new_population.extend([mutate(child1, mutation_std), mutate(child2, mutation_std)])

        population = new_population[:POP_SIZE]  # Ensure population size remains constant
        best_fitness = max(fitnesses)
        print(f"Generation {gen+1}: Best Fitness = {best_fitness:.4f} | Mutation STD = {mutation_std:.4f} | SBX_ETA = {sbx_eta:.2f}")

        mutation_std *= MUTATION_DECAY  # decay mutation over time
        sbx_eta *= SBX_ETA_GROWTH       # increase SBX eta over time (more conservative)

    # Final result
    final_fitnesses = [fitness(ind) for ind in population]
    best = max(zip(population, final_fitnesses), key=lambda x: x[1])
    print("\nBest Individual:", best[0])
    print("Best Fitness:", best[1])

if __name__ == "__main__":
    start_time = time.time()
    genetic_algorithm()
    end_time = time.time()
    print(f"Execution time using Roulette Selection: {end_time - start_time:.4f} seconds")


Generation 1: Best Fitness = 0.0303 | Mutation STD = 0.5000 | SBX_ETA = 2.00
Generation 2: Best Fitness = 0.0347 | Mutation STD = 0.4750 | SBX_ETA = 2.10
Generation 3: Best Fitness = 0.0394 | Mutation STD = 0.4512 | SBX_ETA = 2.21
Generation 4: Best Fitness = 0.0424 | Mutation STD = 0.4287 | SBX_ETA = 2.32
Generation 5: Best Fitness = 0.0445 | Mutation STD = 0.4073 | SBX_ETA = 2.43
Generation 6: Best Fitness = 0.0509 | Mutation STD = 0.3869 | SBX_ETA = 2.55
Generation 7: Best Fitness = 0.0509 | Mutation STD = 0.3675 | SBX_ETA = 2.68
Generation 8: Best Fitness = 0.0509 | Mutation STD = 0.3492 | SBX_ETA = 2.81
Generation 9: Best Fitness = 0.0525 | Mutation STD = 0.3317 | SBX_ETA = 2.95
Generation 10: Best Fitness = 0.0534 | Mutation STD = 0.3151 | SBX_ETA = 3.10
Generation 11: Best Fitness = 0.0562 | Mutation STD = 0.2994 | SBX_ETA = 3.26
Generation 12: Best Fitness = 0.0586 | Mutation STD = 0.2844 | SBX_ETA = 3.42
Generation 13: Best Fitness = 0.0586 | Mutation STD = 0.2702 | SBX_ETA = 

In [4]:
import random
import numpy as np
import math
import time

# === Parameters ===
POP_SIZE = 50
GENES = 8
GENERATIONS = 30
ELITE_COUNT = 3
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.1
INITIAL_MUTATION_STD = 0.5  # High initial value
MUTATION_DECAY = 0.95       # Decay factor per generation
INITIAL_SBX_ETA = 2
SBX_ETA_GROWTH = 1.05       # SBX eta increases each generation

# === Fixed input individuals for blend ===
INPUT_A = [0.9, 0.9, 0.0, 0.8, 0.2, 0.9, 0.7, 0.7]
INPUT_B = [0.8, 0.2, 1.0, 0.3, 0.9, 0.4, 0.6, 0.3]

# === Fixed input individuals for blend ===
# INPUT_A = [1, 0.5, 0.0, 0.4, 1, 0.5, 0.5, 0.4]
# INPUT_B = [0.8, 0.2, 1.0, 0.3, 1, 0.4, 0.6, 0.3]

# === Fitness Function ===
def fitness(candidate):
    emergence = [c - max(a, b) for c, a, b in zip(candidate, INPUT_A, INPUT_B)]
    emergence = [max(0, e) for e in emergence]  # clamp negative emergence to 0
    contributions = [min(a, b) * e for a, b, e in zip(INPUT_A, INPUT_B, emergence)]
    total = sum(contributions)
    return min(total / GENES, 1.0)

# === Initialization ===
def initialize_population():
    return [np.random.uniform(0, 1, GENES).tolist() for _ in range(POP_SIZE)]


# Implemented Tournament selection method instead of Roulette-Wheel
def tournament_selection(population, fitnesses, k=3):
    selected = random.sample(list(zip(population, fitnesses)), k)
    return max(selected, key=lambda x: x[1])[0]

# === Crossover: Simulated Binary Crossover (Adaptive Eta) ===
def sbx_crossover(p1, p2, eta):
    if random.random() > CROSSOVER_RATE:
        return p1[:], p2[:]

    child1, child2 = [], []
    for x1, x2 in zip(p1, p2):
        if random.random() <= 0.5:
            if abs(x1 - x2) > 1e-14:
                x1, x2 = min(x1, x2), max(x1, x2)
                rand = random.random()
                beta = 1.0 + (2.0 * (x1) / (x2 - x1))
                alpha = 2.0 - beta ** -(eta + 1)
                if rand <= 1.0 / alpha:
                    betaq = (rand * alpha) ** (1.0 / (eta + 1))
                else:
                    betaq = (1.0 / (2.0 - rand * alpha)) ** (1.0 / (eta + 1))
                c1 = 0.5 * ((x1 + x2) - betaq * (x2 - x1))
                c2 = 0.5 * ((x1 + x2) + betaq * (x2 - x1))
                child1.append(min(max(c1, 0.0), 1.0))
                child2.append(min(max(c2, 0.0), 1.0))
            else:
                child1.append(x1)
                child2.append(x2)
        else:
            child1.append(x1)
            child2.append(x2)
    return child1, child2

# === Mutation ===
def mutate(individual, mutation_std):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            individual[i] += random.gauss(0, mutation_std)
            individual[i] = min(max(individual[i], 0), 1)  # clip to [0, 1]
    return individual

# === Main GA Loop ===
def genetic_algorithm():
    population = initialize_population()
    mutation_std = INITIAL_MUTATION_STD
    sbx_eta = INITIAL_SBX_ETA

    for gen in range(GENERATIONS):
        fitnesses = [fitness(ind) for ind in population]
        elites = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)[:ELITE_COUNT]
        new_population = [ind for ind, _ in elites]

        while len(new_population) < POP_SIZE:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child1, child2 = sbx_crossover(parent1, parent2, eta=sbx_eta)
            new_population.extend([mutate(child1, mutation_std), mutate(child2, mutation_std)])

        population = new_population[:POP_SIZE]  # Ensure population size remains constant
        best_fitness = max(fitnesses)
        print(f"Generation {gen+1}: Best Fitness = {best_fitness:.4f} | Mutation STD = {mutation_std:.4f} | SBX_ETA = {sbx_eta:.2f}")

        mutation_std *= MUTATION_DECAY  # decay mutation over time
        sbx_eta *= SBX_ETA_GROWTH       # increase SBX eta over time (more conservative)

    # Final result
    final_fitnesses = [fitness(ind) for ind in population]
    best = max(zip(population, final_fitnesses), key=lambda x: x[1])
    print("\nBest Individual:", best[0])
    print("Best Fitness:", best[1])

if __name__ == "__main__":
    start_time = time.time()
    genetic_algorithm()
    end_time = time.time()
    print(f"Execution time using Tournament Selection: {end_time - start_time:.4f} seconds")


Generation 1: Best Fitness = 0.0290 | Mutation STD = 0.5000 | SBX_ETA = 2.00
Generation 2: Best Fitness = 0.0367 | Mutation STD = 0.4750 | SBX_ETA = 2.10
Generation 3: Best Fitness = 0.0414 | Mutation STD = 0.4512 | SBX_ETA = 2.21
Generation 4: Best Fitness = 0.0441 | Mutation STD = 0.4287 | SBX_ETA = 2.32
Generation 5: Best Fitness = 0.0507 | Mutation STD = 0.4073 | SBX_ETA = 2.43
Generation 6: Best Fitness = 0.0553 | Mutation STD = 0.3869 | SBX_ETA = 2.55
Generation 7: Best Fitness = 0.0568 | Mutation STD = 0.3675 | SBX_ETA = 2.68
Generation 8: Best Fitness = 0.0588 | Mutation STD = 0.3492 | SBX_ETA = 2.81
Generation 9: Best Fitness = 0.0588 | Mutation STD = 0.3317 | SBX_ETA = 2.95
Generation 10: Best Fitness = 0.0610 | Mutation STD = 0.3151 | SBX_ETA = 3.10
Generation 11: Best Fitness = 0.0612 | Mutation STD = 0.2994 | SBX_ETA = 3.26
Generation 12: Best Fitness = 0.0612 | Mutation STD = 0.2844 | SBX_ETA = 3.42
Generation 13: Best Fitness = 0.0613 | Mutation STD = 0.2702 | SBX_ETA = 